### !!! THIS FILE IS NOT FUNCTIONAL, BUT SHOWCASES THE PROCESS OF IMAGE CREATION FROM THE ECODATACUBE !!!

In [ ]:
import os
import rioxarray
import rasterio
import numpy as np
from tqdm import tqdm
from pyproj import Transformer

In [ ]:
TIFF_DIR = "sent2_files_2021/"
os.listdir(TIFF_DIR)

color = "rgb"

red = rioxarray.open_rasterio(
    TIFF_DIR
    + "lcv_"
    + "red"
    + "_sentinel.s2l2a_p50_10m_0..0cm_2021.03.21..2021.12.01_eumap_epsg3035_v1.0.tif"
)
red_rio = rasterio.open(
    TIFF_DIR
    + "lcv_"
    + "red"
    + "_sentinel.s2l2a_p50_10m_0..0cm_2021.03.21..2021.12.01_eumap_epsg3035_v1.0.tif"
)
blue = rioxarray.open_rasterio(
    TIFF_DIR
    + "lcv_"
    + "blue"
    + "_sentinel.s2l2a_p50_10m_0..0cm_2021.03.21..2021.12.01_eumap_epsg3035_v1.0.tif"
)
green = rioxarray.open_rasterio(
    TIFF_DIR
    + "lcv_"
    + "green"
    + "_sentinel.s2l2a_p50_10m_0..0cm_2021.03.21..2021.12.01_eumap_epsg3035_v1.0.tif"
)
# nir = rioxarray.open_rasterio(TIFF_DIR + 'lcv_' + "nir" + '_sentinel.s2l2a_p50_10m_0..0cm_2021.03.21..2021.12.01_eumap_epsg3035_v1.0.tif')

tf = Transformer.from_crs("epsg:4326", "epsg:3035")
CLIP = 10000


def get_image_pixels_raster(lon, lat, dir_path=""):

    y, x = tf.transform(lat, lon)
    south, east = red_rio.index(x, y)
    hs = 64
    r = red[0, east - hs : east + hs, south - hs : south + hs]
    g = green[0, east - hs : east + hs, south - hs : south + hs]
    b = blue[0, east - hs : east + hs, south - hs : south + hs]
    try:
        img = np.stack([r, g, b], axis=-1)
        img[img > CLIP] = CLIP
        img = (img - img.min(axis=0).min(axis=0)) / (
            img.max(axis=0).max(axis=0) - img.min(axis=0).min(axis=0)
        )
        np.power(img, 1 / 2.5)
        img = (img * 255).astype(np.uint8)
    except:
        pass

    # Save image


RES_LON = 502
RES_LAT = 408
max_lon = 34.55792
min_lon = -10.53904
max_lat = 71.18392
min_lat = 34.56858

for i in tqdm(range(RES_LON)):
    # i is lon
    # j is lat
    for j in range(RES_LAT):
        lon = i / RES_LON
        lat = j / RES_LAT
        lon = lon * (max_lon - min_lon) + min_lon
        lat = lat * (max_lat - min_lat) + min_lat

        # get_image_pixels_raster(lon, lat)